# Extraction and Analysis over a Fidelity Multi-Fund Annual Report

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/extract/asset_manager_fund_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to create an agentic document workflow over a complex document that contains annual reports for multiple funds - each fund reports financials in a standardized reporting structure, and it's all consolidated in the same document.

We show you how to create a workflow that parses the document, splits it into subsections per fund, and runs extraction per subsection. The extracted results can be compiled into a single CSV file.

![](asset_manager_fund_analysis.png)


## Setup


### Data

We load the consolidated document from Fidelity's Shareholder Report portal.

**To get the original Doc**

Link: https://fundresearch.fidelity.com/prospectus/sec
- Go to row "Fidelity Advisor Asset Manager® 20% Fund Class A", and click "Annual" under "Financial Statements & Other Information"

**Or, just download it below (we're hosting it on Dropbox)**

In [ ]:
!wget "https://www.dropbox.com/scl/fi/bhrtivs7b2gz3yhrr4t4s/fidelity_fund.pdf?rlkey=ha2loufvuer1c07u47k68hgji&st=ev66x31t&dl=1" -O data/asset_manager_fund_analysis/fidelity_fund.pdf

### Everything else

In the below sections we define the required environment variables and LLM/embedding model.

Sign up for a LlamaCloud account here: https://cloud.llamaindex.ai/

In [ ]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [ ]:
# set LlamaCloud project/organization ID
# project_id = None
# organization_id = None
project_id = "2fef999e-1073-40e6-aeb3-1f3c0e64d99b"
organization_id = "43b88c8f-e488-46f6-9013-698e3d2e374a"

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# set LLM, embedding model
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
llm = OpenAI(model="gpt-4.1")
Settings.llm = llm
Settings.embed_model = embed_model

## Create Splitter

The document is a consolidated regulatory filing that contains multiple sub-entities - the 7 different Asset Manager funds - with identical reporting structures.

Our first task is to create a document splitter that can identify the specific page numbers corresponding to each fund. This is a pre-requisite for the downstream task of doing per-fund extraction and consolidation.

The below block uses LlamaParse to parse the full document and output the result in per-page Markdown nodes.

#### Setup Parser

In [ ]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    premium_mode=True,
    result_type="markdown",
    project_id=project_id,
    organization_id=organization_id,
)
result = await parser.aparse("./data/asset_manager_fund_analysis/fidelity_fund.pdf")
markdown_nodes = await result.aget_markdown_nodes(split_by_page=True)

Retrying llama_cloud_services.parse.utils.make_api_request.<locals>._make_request in 4.0 seconds as it raised RemoteProtocolError: Server disconnected without sending a response..
Retrying llama_cloud_services.parse.utils.make_api_request.<locals>._make_request in 4.0 seconds as it raised RemoteProtocolError: Server disconnected without sending a response..


Started parsing the file under job_id 36c97d60-995c-4b82-824b-3553c55d93e3


### Define Splitting Functions

We then define a set of functions to find the "splits". 
1. We first identify the split categories given a user description. (e.g. what are the precise labels we want to split on?) 
2. We then split the document according to the split categories. 

On (2), we do this by using LLMs on each page to detect if a split starts on that page. We then count all pages from the start of the current split to the start on the next split as within the current split.

#### Find Split Categories

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from llama_index.core.schema import TextNode
from llama_index.core.llms import LLM
from llama_index.core.async_utils import run_jobs
from llama_index.core.prompts import ChatPromptTemplate, ChatMessage
from collections import defaultdict


split_category_prompt = """\
You are an AI document assistant tasked with finding the 'split categories' given a user description and the document text.
- The split categories is a list of string tags from the document that correspond to the user description.
- Do not make up split categories. 
- Do not include category tags that don't fit the user description,\
for instance subcategories or extraneous titles.
- Do not exclude category tags that do fit the user description. 

For instance, if the user asks to "find all top-level sections of an ArXiv paper", then a sample output would be:
["1. Introduction", "2. Related Work", "3. Methodology", "4. Experiments", "5. Conclusion"]

The split description and document text are given below. 

Split description:
{split_description}

Here is the document text:
{document_text}
    
"""


class SplitCategories(BaseModel):
    """A list of all split categories from a document."""

    split_categories: List[str]


async def afind_split_categories(
    split_description: str,
    nodes: List[TextNode],
    llm: Optional[LLM] = None,
    page_limit: Optional[int] = 5,
) -> List[str]:
    """Find split categories given a user description and the page limit.
    
    These categories will then be used to find the exact splits of the document. 
    
    NOTE: with the page limit there is an assumption that the categories are found in the first few pages,\
    for instance in the table of contents. This does not account for the case where the categories are \
    found throughout the document. 
    
    """
    llm = llm or OpenAI(model="gpt-4.1")

    chat_template = ChatPromptTemplate(
        [
            ChatMessage.from_str(split_category_prompt, "user"),
        ]
    )
    nodes_head = nodes[:page_limit] if page_limit is not None else nodes
    doc_text = "\n-----\n".join(
        [n.get_content(metadata_mode="all") for n in nodes_head]
    )

    result = await llm.astructured_predict(
        SplitCategories,
        chat_template,
        split_description=split_description,
        document_text=doc_text,
    )
    return result.split_categories

In [ ]:
split_categories = await afind_split_categories(
    "Find and split by the main funds in this document, should be listed in the first few pages",
    markdown_nodes,
    llm=llm,
    page_limit=5,
)

In [ ]:
split_categories

['Fidelity Asset Manager® 20%',
 'Fidelity Asset Manager® 30%',
 'Fidelity Asset Manager® 40%',
 'Fidelity Asset Manager® 50%',
 'Fidelity Asset Manager® 60%',
 'Fidelity Asset Manager® 70%',
 'Fidelity Asset Manager® 85%']

#### Split Document based on Existing Categories/Rules

In [ ]:
# go through each node (on a page level), and tag node


split_prompt = """\
You are an AI document assistant tasked with extracting out splits from a document text according to a certain set of rules. 

You are given a chunk of the document text at a time. 
You are responsible for determining if the chunk of the document text corresponds to the beginning of a split. 

We've listed general rules below, and the user has also provided their own rules to find a split. Please extract
out the splits according to the defined schema. 

General Rules: 
- You should ONLY extract out a split if the document text contains the beginning of a split.
- If the document text contains the beginning of two or more splits (e.g. there are multiple sections on a single page), then \
return all splits in the output.
- If the text does not correspond to the beginning of any split, then return a blank list. 
- A valid split must be clearly delineated in the document text according to the user rules. \
Do NOT identify a split if it is mentioned, but is not actually the start of a split in the document text.
- If you do find one or more splits, please output the split_name according to the format \"{split_key}_X\", \
where X is a short tag corresponding to the split. 

Split key:
{split_key}

User-defined rules:
{split_rules}


Here is the chunk text:
{chunk_text}
    
"""


class SplitOutput(BaseModel):
    """The metadata for a given split start given a chunk."""

    split_name: str = Field(
        ..., description="The name of the split (in the format \{split_key\}_X)"
    )
    split_description: str = Field(
        ..., description="A short description corresponding to the split."
    )
    page_number: int = Field(..., description="Page number of the split.")


class SplitsOutput(BaseModel):
    """A list of all splits given a chunk."""

    splits: List[SplitOutput]


async def atag_splits_in_node(
    split_rules: str, split_key: str, node: TextNode, llm: Optional[LLM] = None
):
    """Tag split in a single node."""
    llm = llm or OpenAI(model="gpt-4o")

    chat_template = ChatPromptTemplate(
        [
            ChatMessage.from_str(split_prompt, "user"),
        ]
    )

    result = await llm.astructured_predict(
        SplitsOutput,
        chat_template,
        split_rules=split_rules,
        split_key=split_key,
        chunk_text=node.get_content(metadata_mode="all"),
    )
    return result.splits


async def afind_splits(
    split_rules: str, split_key: str, nodes: List[TextNode], llm: Optional[LLM] = None
) -> Dict:
    """Find splits."""

    # tag each node with split or no-split
    tasks = [atag_splits_in_node(split_rules, split_key, n, llm=llm) for n in nodes]
    async_results = await run_jobs(tasks, workers=8, show_progress=True)
    all_splits = [s for r in async_results for s in r]

    split_name_to_pages = defaultdict(list)

    split_idx = 0
    for idx, n in enumerate(nodes):
        cur_page = n.metadata["page_number"]

        # update the current split if needed
        while (
            split_idx + 1 < len(all_splits)
            and all_splits[split_idx + 1].page_number <= cur_page
        ):
            split_idx += 1

        # add page number to the current split
        if all_splits[split_idx].page_number <= cur_page:
            split_name = all_splits[split_idx].split_name
            split_name_to_pages[split_name].append(cur_page)

    # print(all_splits)
    # print(split_name_to_pages)
    # raise Exception

    return split_name_to_pages

#### Find Categories and then Run Splits

In [ ]:
# put it all together - detect categories, then split document based on those categories
async def afind_categories_and_splits(
    split_description: str,
    split_key: str,
    nodes: List[TextNode],
    additional_split_rules: Optional[str] = None,
    llm: Optional[LLM] = None,
    page_limit: int = 5,
    verbose: bool = False,
):
    """Find categories and then splits."""
    categories = await afind_split_categories(
        split_description, nodes, llm=llm, page_limit=page_limit
    )
    if verbose:
        print(f"Split categories: {categories}")
    full_split_rules = f"""Please split by these categories: {categories}"""
    if additional_split_rules:
        full_split_rules += f"\n\n\n{additional_split_rules}"

    return await afind_splits(full_split_rules, split_key, nodes, llm=llm)

#### Define document-specific split rules

Now that we have the general splitter, let's define the rules to help parse this Fidelity document.

**NOTE**: Ideally the valid names are automatically identified, through a separate pass in the workflow.

In [ ]:
# fidelity_split_rules = """
# - You must split by the name of the fund
# - Here are the valid names - do not include subcategories:
#     Fidelity Asset Manager® 20%
#     Fidelity Asset Manager® 30%
#     Fidelity Asset Manager® 40%
#     Fidelity Asset Manager® 50%
#     Fidelity Asset Manager® 60%
#     Fidelity Asset Manager® 70%
#     Fidelity Asset Manager® 85%
# - Each fund will have a list of tables underneath it, like schedule of investments, financial statements
# - Each fund usually has schedule of investments right underneath it
# - Do not tag the cover page/table of contents
# """
# fidelity_split_key = "fidelity_asset_manager"


fidelity_split_description = "Find and split by the main funds in this document, should be listed in the first few pages"
fidelity_split_rules = """
- You must split by the name of the fund
- Each fund will have a list of tables underneath it, like schedule of investments, financial statements
- Each fund usually has schedule of investments right underneath it 
- Do not tag the cover page/table of contents
"""
fidelity_split_key = "fidelity_asset_manager"

#### Try it out

In [ ]:
split_name_to_pages = await afind_categories_and_splits(
    fidelity_split_description,
    fidelity_split_key,
    markdown_nodes,
    additional_split_rules=fidelity_split_rules,
    llm=llm,
    verbose=True,
)

Split categories: ['Fidelity Asset Manager® 20%', 'Fidelity Asset Manager® 30%', 'Fidelity Asset Manager® 40%', 'Fidelity Asset Manager® 50%', 'Fidelity Asset Manager® 60%', 'Fidelity Asset Manager® 70%', 'Fidelity Asset Manager® 85%']


100%|████████████████████████████████████████████████████████████████████| 120/120 [00:28<00:00,  4.20it/s]


In [ ]:
split_name_to_pages

defaultdict(list,
            {'fidelity_asset_manager_20': [2,
              3,
              4,
              5,
              6,
              7,
              8,
              9,
              10,
              11,
              12,
              13,
              93,
              94,
              108,
              109,
              110,
              111],
             'fidelity_asset_manager_30': [14,
              15,
              16,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              99,
              101,
              112],
             'fidelity_asset_manager_40': [26,
              27,
              28,
              29,
              30,
              31,
              32,
              33,
              34,
              35,
              36,
              88,
              102,
              106,
              113],
             'fidelity_asse

## Run Per-Fund Extraction, Consolidate Results

We then use LlamaExtract on the aggregated text for each split (page numbers corresponding to each fund), to extract out a Pydantic `FundData` object for each fund.

We then consolidate them together into a CSV.

#### Define Fund Data

In [ ]:
# Define output schema
from pydantic import BaseModel
from typing import Optional


class FundData(BaseModel):
    """Concise fund data extraction schema optimized for LLM extraction"""

    # Identifiers
    fund_name: str = Field(
        ...,
        description="Full fund name exactly as it appears, e.g. 'Fidelity Asset Manager® 20%'",
    )
    target_equity_pct: Optional[int] = Field(
        None,
        description="Target equity percentage from fund name (20, 30, 40, 50, 60, 70, or 85)",
    )
    report_date: Optional[str] = Field(
        None, description="Report date in YYYY-MM-DD format, e.g. '2024-09-30'"
    )

    # Asset Allocation (as percentages, e.g. 27.4 for 27.4%)
    equity_pct: Optional[float] = Field(
        None,
        description="Actual equity allocation percentage from 'Equity Central Funds' section",
    )
    fixed_income_pct: Optional[float] = Field(
        None,
        description="Fixed income allocation percentage from 'Fixed-Income Central Funds' section",
    )
    money_market_pct: Optional[float] = Field(
        None,
        description="Money market allocation percentage from 'Money Market Central Funds' section",
    )
    other_pct: Optional[float] = Field(
        None,
        description="Other investments percentage (Treasury + Investment Companies + other)",
    )

    # Primary Share Class Metrics (use the main retail class, usually named after the fund)
    nav: Optional[float] = Field(
        None,
        description="Net Asset Value per share for the main retail class (e.g. Asset Manager 20% class)",
    )
    net_assets_usd: Optional[float] = Field(
        None,
        description="Total net assets in USD for the main retail class from 'Net Asset Value' section",
    )
    expense_ratio: Optional[float] = Field(
        None,
        description="Expense ratio as percentage (e.g. 0.48 for 0.48%) from Financial Highlights",
    )
    management_fee: Optional[float] = Field(
        None,
        description="Management fee rate as percentage from Financial Highlights or Notes",
    )

    # Performance (as percentages)
    one_year_return: Optional[float] = Field(
        None,
        description="One-year total return percentage from Financial Highlights (e.g. 13.74 for 13.74%)",
    )
    portfolio_turnover: Optional[float] = Field(
        None, description="Portfolio turnover rate percentage from Financial Highlights"
    )

    # Risk Metrics (in USD)
    equity_futures_notional: Optional[float] = Field(
        None,
        description="Net notional amount of equity futures contracts (positive if net long, negative if net short)",
    )
    bond_futures_notional: Optional[float] = Field(
        None,
        description="Net notional amount of bond/treasury futures contracts (positive if net long, negative if net short)",
    )

    # Fund Flows (in USD)
    net_investment_income: Optional[float] = Field(
        None,
        description="Net investment income for the period from Statement of Operations",
    )
    total_distributions: Optional[float] = Field(
        None,
        description="Total distributions to shareholders from Statement of Changes in Net Assets",
    )
    net_asset_change: Optional[float] = Field(
        None,
        description="Net change in assets from beginning to end of period (end minus beginning net assets)",
    )


class FundComparisonData(BaseModel):
    """Flattened data optimized for CSV export and analysis"""

    funds: list[FundData]

    def to_csv_rows(self) -> list[dict]:
        """Convert to list of dictionaries for CSV export"""
        return [fund.dict() for fund in self.funds]

#### Setup LlamaExtract 

In [ ]:
from llama_cloud_services import LlamaExtract
from llama_cloud.core.api_error import ApiError
from llama_cloud import ExtractConfig


# Optionally, add your project id/organization id
llama_extract = LlamaExtract(
    show_progress=True,
    check_interval=5,
    project_id=project_id,
    organization_id=organization_id,
)


try:
    existing_agent = llama_extract.get_agent(name="FundDataExtractor2")
    if existing_agent:
        print("deleting existing agent")
        # Deletion can take some time since all underlying files will be purged
        llama_extract.delete_agent(existing_agent.id)
except ApiError as e:
    if e.status_code == 404:
        pass
    else:
        raise

deleting existing agent


In [ ]:
extract_config = ExtractConfig(
    extraction_mode="BALANCED",
)

extract_agent = llama_extract.create_agent(
    "FundDataExtractor2", data_schema=FundData, config=extract_config
)

#### Define Extraction Functions

In [ ]:
from llama_cloud_services.extract import SourceText
from typing import List, Optional, Dict


async def aextract_data_over_split(
    split_name: str,
    page_numbers: List[int],
    nodes: List[TextNode],
    llm: Optional[LLM] = None,
) -> FundData:
    """Extract fund data for a given split."""

    # combine node text that matches the page numbers
    filtered_nodes = [n for n in nodes if n.metadata["page_number"] in page_numbers]
    filtered_text = "\n-------\n".join(
        [n.get_content(metadata_mode="all") for n in filtered_nodes]
    )
    result_dict = (
        await extract_agent.aextract(SourceText(text_content=filtered_text))
    ).data

    fund_data = FundData.model_validate(result_dict)

    return fund_data


async def aextract_data_over_splits(
    split_name_to_pages: Dict[str, List],
    nodes: List[TextNode],
    llm: Optional[LLM] = None,
):
    """Extract fund data for each split, aggregate."""
    tasks = [
        aextract_data_over_split(split_name, page_numbers, nodes, llm=llm)
        for split_name, page_numbers in split_name_to_pages.items()
    ]
    all_fund_data = await run_jobs(tasks, workers=8, show_progress=True)
    return FundComparisonData(funds=all_fund_data)

#### Try it out

In [ ]:
all_fund_data = await aextract_data_over_splits(
    split_name_to_pages, markdown_nodes, llm=llm
)

In [ ]:
import pandas as pd

all_fund_data_df = pd.DataFrame(all_fund_data.to_csv_rows())
all_fund_data_df

,fund_name,target_equity_pct,report_date,equity_pct,fixed_income_pct,money_market_pct,other_pct,nav,net_assets_usd,expense_ratio,management_fee,one_year_return,portfolio_turnover,equity_futures_notional,bond_futures_notional,net_investment_income,total_distributions,net_asset_change
0,Fidelity Asset Manager® 20%,20,2024-09-30,27.4,48.6,16.0,8.0,13.95,3.170882e+09,0.48,0.43,13.74,24.0,-308348.0,159650906.0,180771003.0,1.815011e+08,148593411.0
1,Fidelity Asset Manager® 30%,30,2024-09-30,37.4,48.4,6.3,8.1,12.21,1.375227e+09,0.49,0.44,16.20,19.0,-117123430.0,60683344.0,60219520.0,6.074714e+07,61425310.0
2,Fidelity Asset Manager® 40%,40,2024-09-30,47.3,43.3,6.4,3.0,13.40,1.544767e+09,0.51,NaN,18.24,13.0,-131196445.0,68340188.0,59239637.0,3.794509e+07,117313232.0
3,Fidelity Asset Manager® 50%,50,2024-09-30,57.1,37.3,2.1,3.5,21.44,7.407487e+09,0.56,0.58,20.37,12.0,-550922128.0,288445875.0,223428532.0,2.197314e+08,771033500.0
4,Fidelity Asset Manager® 60%,60,2024-09-30,63.8,32.2,0.7,3.3,16.32,2.282371e+09,0.64,0.54,22.31,17.0,-103026215.0,127423594.0,84848944.0,7.662887e+07,630046421.0
5,Fidelity Asset Manager® 70%,70,2024-09-30,73.7,22.3,1.0,3.3,29.09,4.273012e+09,0.64,0.54,24.06,14.0,-162028290.0,202620656.0,119678101.0,1.121974e+08,935150295.0
6,Fidelity Asset Manager® 85%,85,2024-09-30,84.6,11.2,1.9,3.3,27.85,4.394805e+09,0.94,0.63,27.23,16.0,105407845.0,129202345.0,60293654.0,3.695516e+08,935099874.0


## Define Full Workflow

We put everything together into a LlamaIndex workflow! 

In [ ]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step,
)


class ParseDocEvent(Event):
    nodes: List[TextNode]


class DocSplitEvent(Event):
    split_name_to_pages: Dict[str, List[int]]
    nodes: List[TextNode]


class FidelityFundExtraction(Workflow):
    """
    Workflow to extract data from a solar panel datasheet and generate a comparison report
    against provided design requirements.
    """

    def __init__(
        self,
        parser: LlamaParse,
        extract_agent: LlamaExtract,
        split_description: str = fidelity_split_description,
        split_rules: str = fidelity_split_rules,
        split_key: str = fidelity_split_key,
        llm: Optional[LLM] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.parser = parser
        self.extract_agent = extract_agent
        self.split_description = split_description
        self.split_rules = split_rules
        self.split_key = split_key
        self.llm = llm

    @step
    async def parse_doc(self, ctx: Context, ev: StartEvent) -> ParseDocEvent:
        """Parse document into markdown nodes."""
        result = await parser.aparse(file_path=ev.file_path)
        markdown_nodes = await result.aget_markdown_nodes(split_by_page=True)
        return ParseDocEvent(nodes=markdown_nodes)

    @step
    async def find_splits(self, ctx: Context, ev: ParseDocEvent) -> DocSplitEvent:
        split_name_to_pages = await afind_categories_and_splits(
            self.split_description,
            self.split_key,
            ev.nodes,
            additional_split_rules=self.split_rules,
            llm=llm,
            verbose=True,
        )
        return DocSplitEvent(
            split_name_to_pages=split_name_to_pages,
            nodes=ev.nodes,
        )

    @step
    async def run_extraction(self, ctx: Context, ev: DocSplitEvent) -> StopEvent:
        all_fund_data = await aextract_data_over_splits(
            ev.split_name_to_pages, ev.nodes, llm=self.llm
        )
        all_fund_data_df = pd.DataFrame(all_fund_data.to_csv_rows())
        return StopEvent(
            result={
                "all_fund_data": all_fund_data,
                "all_fund_data_df": all_fund_data_df,
            }
        )

In [ ]:
workflow = FidelityFundExtraction(
    parser=parser, extract_agent=extract_agent, verbose=True, timeout=None
)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    FidelityFundExtraction,
    filename="fidelity_fund_extraction.html",
)

fidelity_fund_extraction.html


In [ ]:
result = await workflow.run(
    file_path="./data/asset_manager_fund_analysis/fidelity_fund.pdf"
)

In [ ]:
all_fund_data_df = result["all_fund_data_df"]

In [ ]:
all_fund_data_df

,fund_name,target_equity_pct,report_date,equity_pct,fixed_income_pct,money_market_pct,other_pct,nav,net_assets_usd,expense_ratio,management_fee,one_year_return,portfolio_turnover,equity_futures_notional,bond_futures_notional,net_investment_income,total_distributions,net_asset_change
0,Fidelity Asset Manager® 20%,20,2024-09-30,27.4,48.6,16.0,8.0,13.95,3.170882e+09,0.48,0.48,13.74,24.0,-30834897.0,159650906.0,180771003.0,181501111.0,148593411.0
1,Fidelity Asset Manager® 30%,30,2024-09-30,37.4,48.4,6.3,8.1,12.21,1.375227e+09,0.49,0.44,16.20,19.0,-117123430.0,60683344.0,60219520.0,60747136.0,61325310.0
2,Fidelity Asset Manager® 40%,40,2024-09-30,47.3,43.3,6.4,3.0,13.40,1.544767e+09,0.51,0.43,18.24,13.0,-131196445.0,68340188.0,59239637.0,-37925052.0,117313232.0
3,Fidelity Asset Manager® 50%,50,2024-09-30,57.1,37.3,2.1,3.5,21.44,7.407487e+09,0.56,0.53,20.37,12.0,-550922128.0,288445875.0,223428532.0,219731373.0,771033500.0
4,Fidelity Asset Manager® 60%,60,2024-09-30,63.8,32.2,0.7,3.3,16.32,2.282371e+09,0.64,0.54,22.31,17.0,-103026215.0,127423594.0,84848944.0,76628870.0,630046421.0
5,Fidelity Asset Manager® 70%,70,2024-09-30,73.7,22.3,1.0,3.3,29.09,4.273012e+09,0.64,0.16,24.06,14.0,-162028290.0,202620656.0,119678101.0,112197413.0,935150295.0
6,Fidelity Asset Manager® 85%,85,2024-09-30,84.6,11.2,1.9,3.3,27.85,4.394805e+09,0.94,0.52,27.23,16.0,104412803.0,129202345.0,60293654.0,51390318.0,935099874.0


## Run Analysis over Compiled Data

Now that we've created an aggregated dataframe from this report, we can now do good ol' Pandas analysis or use LLMs to help do text-to-CSV over it! 

#### Standard Pandas analysis

The conservative 20% fund is actually the most return-efficient.

In [ ]:
# Calculate return per unit of risk (equity allocation)
all_fund_data_df["return_per_risk"] = (
    all_fund_data_df["one_year_return"] / all_fund_data_df["equity_pct"]
)
all_fund_data_df["return_per_risk"]

0    0.501460
1    0.433155
2    0.385624
3    0.356743
4    0.349687
5    0.326459
6    0.321868
Name: return_per_risk, dtype: float64

The lower equity allocation funds consistently run higher than their target.

In [ ]:
# How far do actual allocations drift from targets?
all_fund_data_df["drift"] = (
    all_fund_data_df["equity_pct"] - all_fund_data_df["target_equity_pct"]
)
all_fund_data_df["drift"]

0    7.4
1    7.4
2    7.3
3    7.1
4    3.8
5    3.7
6   -0.4
Name: drift, dtype: float64

#### Text-to-Pandas Analysis

In [ ]:
from llama_index.experimental.query_engine import PandasQueryEngine

In [ ]:
pd_query_engine = PandasQueryEngine(
    df=all_fund_data_df, verbose=True, synthesize_response=True
)

In [ ]:
response = pd_query_engine.query(
    "Show me all funds where the actual equity allocation is more than 5% higher than the target"
)
print(str(response))

> Pandas Instructions:
```
df[df['equity_pct'] > df['target_equity_pct'] + 5]
```
> Pandas Output:                      fund_name  target_equity_pct report_date  equity_pct  \
0  Fidelity Asset Manager® 20%                 20  2024-09-30        27.4   
1  Fidelity Asset Manager® 30%                 30  2024-09-30        37.4   
2  Fidelity Asset Manager® 40%                 40  2024-09-30        47.3   
3  Fidelity Asset Manager® 50%                 50  2024-09-30        57.1   

   fixed_income_pct  money_market_pct  other_pct    nav  net_assets_usd  \
0              48.6              16.0        8.0  13.95    3.170882e+09   
1              48.4               6.3        8.1  12.21    1.375227e+09   
2              43.3               6.4        3.0  13.40    1.544767e+09   
3              37.3               2.1        3.5  21.44    7.407487e+09   

   expense_ratio  management_fee  one_year_return  portfolio_turnover  \
0           0.48            0.48            13.74                2